In [28]:
import pandas as pd
from datetime import timedelta

def analyze_file(file_path):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Convert the 'Time' and 'Time Out' columns to datetime objects, handling different date formats
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
    df['Time Out'] = pd.to_datetime(df['Time Out'], errors='coerce')

    # Sort the DataFrame by 'Employee Name' and 'Time' for easier analysis
    df.sort_values(by=['Employee Name', 'Time'], inplace=True)

    # Initialize variables to track consecutive days and previous time out
    consecutive_days = 1
    prev_time_out = None

    # Initialize sets to store unique results for each condition
    employees_more_than_7_days = set()
    employees_more_than_14_hours = set()
    employees_less_than_10_hours = set()

    # Open a file for writing
    with open('Output.txt', 'w') as output_file:
        # Iterate through the DataFrame to analyze the data
        for index, row in df.iterrows():
            current_time = row['Time']
            current_time_out = row['Time Out']

            # Check for consecutive days based on different calendar dates
            if not pd.isna(prev_time_out) and not pd.isna(current_time):
                if current_time.date() - prev_time_out.date() == timedelta(days=1):
                    consecutive_days += 1
                else:
                    consecutive_days = 1

            # a) Employees who have worked for 7 consecutive days
            if consecutive_days == 7:
                print(f"Employee: {row['Employee Name']}, Position: {row['Position ID']}", file=output_file)
                employees_more_than_7_days.add((row['Employee Name'], row['Position ID']))

            # b) Employees with less than 10 hours between shifts but greater than 1 hour
            if not pd.isna(prev_time_out) and not pd.isna(current_time) and (current_time - prev_time_out) < timedelta(hours=10) and (current_time - prev_time_out) > timedelta(hours=1):
                print(f"Employee: {row['Employee Name']}, Position: {row['Position ID']}, Reason: Less than 10 hours between shifts", file=output_file)
                employees_less_than_10_hours.add((row['Employee Name'], row['Position ID']))

            # c) Employees who have worked for more than 14 hours in a single shift
            if not pd.isna(current_time) and not pd.isna(current_time_out) and (current_time_out - current_time) > timedelta(hours=14):
                print(f"Employee: {row['Employee Name']}, Position: {row['Position ID']}, Reason: Worked for more than 14 hours in a single shift", file=output_file)
                employees_more_than_14_hours.add((row['Employee Name'], row['Position ID']))

            # Update the previous time out for the next iteration
            prev_time_out = current_time_out

    # Display additional results (optional)
    # print("\nEMPLOYEES WHO HAVE WORKED FOR MORE THAN 14 HOURS IN A SINGLE SHIFT:")
    # for employee in employees_more_than_14_hours:
    #     print(f"Employee: {employee[0]}, Position: {employee[1]}")

    # print("\nEMPLOYEES WITH LESS THAN 10 HOURS BETWEEN SHIFTS BUT GREATER THAN 1 HOUR:")
    # for employee in employees_less_than_10_hours:
    #     print(f"Employee: {employee[0]}, Position: {employee[1]}")

if __name__ == "__main__":
    file_path = "Assignment_Timecard.csv"  # Replace with the actual file path
    analyze_file(file_path)
